# Filtrado colaborativo usuario a usuario

**Autor**: Arturo Sánchez Palacio

Basado en: https://github.com/lazyprogrammer

**Fecha de última revisión: 16/I/2020**

Este notebook se emplearán los siguientes módulos:

In [1]:
import os
import pickle
import numpy as np

Comenzamos cargando los datos almacenados en los diccionarios generados en el preprocesamiento:

__Nota.__ Si el notebook de preprocesamiento no se ha ejecutado antes debe ejecutarse ahora para generar los datos con los que se trabaja en este notebook:

In [2]:
with open('./data/usuario_pelicula.json', 'rb') as f:
  usuario_pelicula = pickle.load(f)

with open('./data/pelicula_usuario.json', 'rb') as f:
  pelicula_usuario = pickle.load(f)

with open('./data/usuariopeli_rating.json', 'rb') as f:
  usuariopeli_rating = pickle.load(f)

with open('./data/usuariopeli_rating_test.json', 'rb') as f:
  usuariopeli_rating_test = pickle.load(f)

De nuevo vamos a cálcular N y M como el número máximo de usuarios películas. Esta vez es algo más complicado pues puede haber películas que no aparezcan en el training set pero sí en el test set. Esto mismo es muy improbable para usuarios por lo que N se puede calcular de manera trivial:

#### To do. Calcular el número de usuarios:

Para calcular M buscamos el máximo tanto en pelicula_usuario como en usuariopeli_rating_test y fijamos como M el máximo de ambas:

In [3]:
#Usuario
N = np.max(list(usuario_pelicula.keys())) + 1


M train: 999
M train: 999


#### To do calcular el número de películas:

In [5]:
#Películas
m_train = np.max(list(pelicula_usuario.keys()))
m_test = np.max([m for (u, m), r in usuariopeli_rating_test.items()])
M = max(m_train, m_test)

print("M train: " + str(m_train))
print("M test: " + str(m_test))

M train: 999
M test: 999


In [6]:
print(M,N)

999 2500


Aquí reflexionamos de nuevo sobre el coste computacional de la operación. Recordemos que el algoritmo es  $O(N^2 * M)$ así que para valores de N muy altos deberíamos plantearnos volver a muestrear. 10.000 es elevado pero está bien así que pododemos continuar:

__Nota.__ Recordemos las fórmulas que definen los pesos y las predicciones:

![title](media/pesos.png)

![title](media/predicciones.png)

Como discutimos previamente en la explicación teórica fijamos el número de vecinos máximo que vamos a considerar y un umbral de películas comunes (no calcularemos pesos para usuarios que tengan menos películas en común que este umbral). Fijamos explorar 25 vecinos (como máximo) y un umbral de al menos cinco películas en común con el usuario:

In [7]:
K = 25 #número máximo de vecinos
umbral = 5 #número mínimo de películas comunes

Construimos tres listas vacías en las que almacenaremos:

In [8]:
vecinos = [] #vecinos para el usuario
medias = [] # valoración media de cada usuario
desviaciones = [] #desviaciones de cada usuario

Para cada usuario:

In [9]:
from sortedcontainers import SortedList
for i in range(N):
    #buscamos los K usuarios más próximos a i
    peliculas_i = usuario_pelicula[i] #pelis que el usuario ha visto
    peliculas_i_set = set(peliculas_i) #las almaceno en un conjunto
    ratings_i = { pelicula:usuariopeli_rating[(i, pelicula)] for pelicula in peliculas_i } #construyo un diccionario de clave película y valor sus puntaciones
    avg_i = np.mean(list(ratings_i.values())) #puntuación media del usuario i
    dev_i = {pelicula:(rating - avg_i) for pelicula, rating in ratings_i.items()} #desviaciones del usuario i
    dev_i_values = np.array(list(dev_i.values())) #array de Numpy con las desviaciones
    sigma_i = np.sqrt(dev_i_values.dot(dev_i_values)) #denominador del coeficiente de Pearson

    medias.append(avg_i) #guardo las medias para usarlas en un futuro
    desviaciones.append(dev_i) #guardo las desviaciones para usarlas en un futuro

    sl = SortedList() #Creo una lista ordenada en la que almacenar los pesos que ya tengo calculados (mantengo las 25 entradas que nos interesan)
                         
    for j in range(N): #Para cada usuario distinto de i calculo el peso i j (no es eficiente computacionalmente pero costaría más memoria optimizarlo)
    
        if j != i: #el propio usuario no puede ser su propio vecino
            peliculas_j = usuario_pelicula[j]
            peliculas_j_set = set(peliculas_j)
            peliculas_comunes = (peliculas_i_set & peliculas_j_set) # peliculas comunes entre i y j (intersección)
            if len(peliculas_comunes) > umbral: #solo calculamos el peso si son suficientemente similares
        
                ratings_j = { pelicula:usuariopeli_rating[(j, pelicula)] for pelicula in peliculas_j } #diccionario
                avg_j = np.mean(list(ratings_j.values())) #puntuación media
                dev_j = { pelicula:(rating - avg_j) for pelicula, rating in ratings_j.items() } #desviaciones
                dev_j_values = np.array(list(dev_j.values())) #array de Numpy con las desviaciones
                sigma_j = np.sqrt(dev_j_values.dot(dev_j_values)) #denominador del coeficiente de Pearson

                # calculate correlation coefficient
                numerador = sum(dev_i[m]*dev_j[m] for m in peliculas_comunes)
                w_ij = numerador / (sigma_i * sigma_j)

                sl.add((-w_ij, j)) #añadimos el peso y el usuario a la lista ordenada (- porque ordena ascendentemente)
                if len(sl) > K: # Si la lista supera el límite de vecinos eliminamos el peso menos importante
                    del sl[-1]

    
    vecinos.append(sl) #almacenamos los vecinos (usuario 0 en posición 0...)
    print(i,"/",N)

0 / 2500
1 / 2500
2 / 2500
3 / 2500
4 / 2500
5 / 2500
6 / 2500
7 / 2500
8 / 2500
9 / 2500
10 / 2500
11 / 2500
12 / 2500
13 / 2500
14 / 2500
15 / 2500
16 / 2500
17 / 2500
18 / 2500
19 / 2500
20 / 2500
21 / 2500
22 / 2500
23 / 2500
24 / 2500
25 / 2500
26 / 2500
27 / 2500
28 / 2500
29 / 2500
30 / 2500
31 / 2500
32 / 2500
33 / 2500
34 / 2500
35 / 2500
36 / 2500
37 / 2500
38 / 2500
39 / 2500
40 / 2500
41 / 2500
42 / 2500
43 / 2500
44 / 2500
45 / 2500
46 / 2500
47 / 2500
48 / 2500
49 / 2500
50 / 2500
51 / 2500
52 / 2500
53 / 2500
54 / 2500
55 / 2500
56 / 2500
57 / 2500
58 / 2500
59 / 2500
60 / 2500
61 / 2500
62 / 2500
63 / 2500
64 / 2500
65 / 2500
66 / 2500
67 / 2500
68 / 2500
69 / 2500
70 / 2500
71 / 2500
72 / 2500
73 / 2500
74 / 2500
75 / 2500
76 / 2500
77 / 2500
78 / 2500
79 / 2500
80 / 2500
81 / 2500
82 / 2500
83 / 2500
84 / 2500
85 / 2500
86 / 2500
87 / 2500
88 / 2500
89 / 2500
90 / 2500
91 / 2500
92 / 2500
93 / 2500
94 / 2500
95 / 2500
96 / 2500
97 / 2500
98 / 2500
99 / 2500
100 / 2500

A continuación construimos una función para calcular las predicciones. Partiendo de un usuario i y una película m predice la puntuación que este usuario daría a la película:

In [10]:
N

2500

In [11]:
"""
i: usuario
m: película a la que el usuario no ha dado su rating
"""
def predict(i, m):
   
    numerador = 0 #suma del producto de los pesos y las desviaciones
    denominador = 0 #suma de los valores absolutos de los pesos
    for vecino in vecinos[i]:
        neg_w = vecino[0]
        j = vecino[1]
        desviaciones_j = desviaciones[j]
        try:
        # Si el vecino ha valorado la película calculamos:
        ###EJERCICIO EMPIEZA
            numerador += (-1)*neg_w*desviaciones_j[m] #Recordar que almacenamos el peso en negativo
            denominador += abs(neg_w*desviaciones_j[m])
        ###EJERCICIO ACABA
        except KeyError:
        #El vecino puede no haber valorado la película que predecimos.
        #Lanzaremos una excepción cuando buscamos en el diccionario y no encontramos la valoración.
            pass

    if denominador == 0:
        prediccion = medias[i] #No podemos hacer nada mejor que esto
    else:
        prediccion = numerador / denominador + medias[i]
    ###EJERCICIO EMPIEZA
     # la fórmula no está acotada así que la acotamos manualmente (esto es peligroso)
        prediccion = min(5, prediccion)
     # la fórmula no está acotada así que la acotamos manualmente (esto es peligroso)
        prediccion = max(0, prediccion)
    ###EJERCICIO ACABA
    return prediccion

Creamos dos listas para almacenar las predicciones y etiquetas:

In [12]:
predicciones_train = []
etiquetas_train = []

#### To do. Realizamos las predicciones para cada película y usuario en entrenamiento:

In [13]:
for (i, m), etiqueta in usuariopeli_rating.items():
    #Calculamos la predicción para la película
    prediction = predict(i, m)
    # Almacenamos la predicción y la etiqueta
    predicciones_train.append(prediction)
    etiquetas_train.append(etiqueta)



#### To do. Análogamente en el test:

In [14]:
predicciones_test = []
etiquetas_test = []

for (i, m), etiqueta in usuariopeli_rating_test.items():
    #Calculamos la predicción para la película
    prediction = predict(i, m)
    # Almacenamos la predicción y la etiqueta
    predicciones_test.append(prediction)
    etiquetas_test.append(etiqueta)

#### To do. Implementamos una función trivial para calcular el error medio cuadrático:

In [15]:
def mse(p, t):
    p = np.array(p)
    t = np.array(t)
    
    return np.mean((p - t)**2)

Podemos observar el error final:

In [16]:
print('train mse:', mse(predicciones_train, etiquetas_train))
print('test mse:', mse(predicciones_test, etiquetas_test))

train mse: 0.6322567055717109
test mse: 0.7050474125686619


## Conclusión

Para ser una aproximación tan naïve los resultados no son nada despreciables. Es poco eficaz computacionalmente como hemos podido observar. Sin embargo es interesante comparar este resultado con los que obtendremos más adelante.